In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import os
import talib as tl
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper
%matplotlib inline

In [2]:
def fills(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

In [3]:
aapl = fills(pdr.get_data_google('MSFT', '2000-01-01', '2018-01-01'))
spy = fills(pdr.get_data_google('SPY', '2000-01-01', '2018-01-01'))

In [4]:
oopen = aapl['Open'].values
high = aapl['High'].values
low = aapl['Low'].values
close = aapl['Close'].values

In [5]:
runup252 = aapl['Close'].pct_change(252)
aapl['Run Up 252'] = runup252

In [6]:
beta63 = tl.BETA(close, spy['Close'].values, timeperiod=63)
aapl['Beta 63'] = beta63

In [7]:
ema100 = tl.EMA(close, timeperiod=100)
aapl['EMA 100'] = ema100
aapl['EMA 100'] = aapl['EMA 100'].pct_change()

In [8]:
sma100 = tl.SMA(close, timeperiod=100)
aapl['SMA 100'] = sma100
aapl['SMA 100'] = aapl['SMA 100'].pct_change()

In [9]:
sma_mom100 = aapl['SMA 100'] - aapl['SMA 100'].shift(1)
aapl['SMA MOM 100'] = sma_mom100

In [10]:
sp500_sma100 = tl.SMA(spy['Close'].values, timeperiod=100)
aapl['SP500 SMA 100'] = sp500_sma100
aapl['SP500 SMA 100'] = aapl['SP500 SMA 100'].pct_change()

In [11]:
sp500vola = tl.ATR(spy['High'].values, spy['Low'].values, spy['Close'].values, timeperiod=63)
aapl['SP500 Vola 63'] = sp500vola

In [12]:
sharpe_days = 63
daily_ret = aapl['Close'].pct_change()
sharpe63 = np.sqrt(sharpe_days)*(daily_ret.rolling(sharpe_days).mean()/daily_ret.rolling(sharpe_days).std())
aapl['Sharpe 63'] = sharpe63

In [13]:
vola63 = tl.ATR(high, low, close, timeperiod=63)
aapl['Vola 63'] = vola63

In [14]:
return_days = 2
ret = 100*aapl['Close'].pct_change(return_days).shift(-return_days)
aapl[str(return_days) + ' days future return'] = ret

In [15]:
aapl['Weekday'] = aapl.index.dayofweek.map(lambda x: {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}.get(x))

In [16]:
def to_class(val, extreme):
    if val < -extreme:
        return 'Neg'
    elif val > extreme:
        return 'Pos'
    else:
        return 'Neutral'
aapl['Return class'] = aapl[str(return_days) + ' days future return'].apply(lambda val: to_class(val, 2))
aapl['Return class'].head()

Date
2002-02-11    Neutral
2002-02-12        Pos
2002-02-13        Neg
2002-02-14        Neg
2002-02-15    Neutral
Name: Return class, dtype: object

In [17]:
aapl.tail(6)

,Open,High,Low,Close,Volume,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63,2 days future return,Weekday,Return class
Date,,,,,,,,,,,,,,,,,
2017-12-21,86.05,86.10,85.40,85.50,17990745,0.345609,0.149057,0.001601,0.001653,0.000010,0.000797,1.454152,1.478645,1.111974,-0.116959,Thu,Neutral
2017-12-22,85.40,85.63,84.92,85.51,14145841,0.345555,0.148297,0.001570,0.001692,0.000039,0.000789,1.442816,1.667317,1.105593,0.233891,Fri,Neutral
2017-12-26,85.31,85.53,85.03,85.40,9891237,0.350411,0.149001,0.001509,0.001690,-0.000003,0.000795,1.429915,1.653098,1.095981,0.374707,Tue,Neutral
2017-12-27,85.65,85.98,85.22,85.71,14678025,0.354456,0.147672,0.001554,0.001659,-0.000031,0.000782,1.418646,1.609473,1.090648,-0.198343,Wed,Neutral
2017-12-28,85.90,85.93,85.55,85.72,10594344,0.360851,0.147436,0.001523,0.001693,0.000034,0.000785,1.405652,1.607795,1.079368,NaN,Thu,Neutral
2017-12-29,85.63,86.05,85.50,85.54,18717406,0.359936,0.148413,0.001446,0.001618,-0.000075,0.000768,1.413657,1.499192,1.070965,NaN,Fri,Neutral


In [18]:
aapl.drop(['Open', 'High', 'Low', 'Close', 'Volume', str(return_days) + ' days future return'], axis=1, inplace=True)
aapl.dropna(inplace=True)

### Resample umbalanced data and Train-test split

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [20]:
train_index = int(len(aapl) * 0.75)
aapl_train = aapl.iloc[:train_index]
aapl_test = aapl.iloc[train_index:]

In [21]:
df_majority = aapl_train[aapl_train['Return class']=='Neutral']
df_minority = aapl_train[aapl_train['Return class']!='Neutral']

df_minority_upsampled = resample(df_minority, 
                                 replace=True,
                                 n_samples=len(df_majority)*2)

aapl_train = pd.concat([df_majority, df_minority_upsampled])
 
aapl_train['Return class'].value_counts()

Pos        2166
Neutral    2017
Neg        1868
Name: Return class, dtype: int64

In [22]:
X_train = aapl_train.drop('Return class', axis=1)
X_test = aapl_test.drop('Return class', axis=1)
y_train = aapl_train['Return class']
y_test = aapl_test['Return class']

In [23]:
trafos = [([col], StandardScaler()) for col in aapl.columns[:-2].tolist()]
trafos = [('Weekday', LabelBinarizer())] + trafos
#trafos += [('Return class', LabelBinarizer())]

In [24]:
trafos

[('Weekday', LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)),
 (['Run Up 252'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Beta 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['EMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SMA MOM 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 SMA 100'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['SP500 Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Sharpe 63'], StandardScaler(copy=True, with_mean=True, with_std=True)),
 (['Vola 63'], StandardScaler(copy=True, with_mean=True, with_std=True))]

In [25]:
mapper = DataFrameMapper(trafos, df_out=True, default=None)

In [26]:
X_train = mapper.fit_transform(X_train)
X_test = mapper.transform(X_test)
X_train.head()

,Weekday_Fri,Weekday_Mon,Weekday_Thu,Weekday_Tue,Weekday_Wed,Run Up 252,Beta 63,EMA 100,SMA 100,SMA MOM 100,SP500 SMA 100,SP500 Vola 63,Sharpe 63,Vola 63
Date,,,,,,,,,,,,,,
2003-02-11,0.0,0.0,0.0,1.0,0.0,-1.273883,0.018267,-1.418923,-0.154516,-1.007615,-0.366097,0.211442,-1.206863,1.421160
2003-02-14,1.0,0.0,0.0,0.0,0.0,-1.157065,-0.031931,-0.872746,0.403333,2.529142,-0.209891,0.190803,-0.971810,1.346634
2003-02-19,0.0,0.0,0.0,0.0,1.0,-0.917909,0.062387,-0.653704,0.336642,-1.209367,0.247836,0.186420,-1.015805,1.337405
2003-02-20,0.0,0.0,1.0,0.0,0.0,-1.049975,0.091262,-0.834976,0.280542,-0.306672,0.033558,0.170051,-1.019991,1.328931
2003-02-21,1.0,0.0,0.0,0.0,0.0,-0.849336,0.031963,-0.583883,0.519364,1.351166,-0.158237,0.159572,-0.772474,1.349826


# ML model

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\DEV\Anaconda3\envs\trading\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [28]:
def fit_and_print(grid):
    grid.fit(X_train, y_train)
    best_estimator = grid.best_estimator_
    print('Best estimator:\n', str(best_estimator))
    
    y_pred = best_estimator.predict(X_test)
    print('Accuracy score:', str(accuracy_score(y_test, y_pred)))
    print('Classification report\n', classification_report(y_test, y_pred))
    print('Confusion matrix\n', str(confusion_matrix(y_test, y_pred)))

### K nearest

In [29]:
params = {'n_neighbors': np.arange(5,50,3),
          'metric': ['euclidean', 'l2', 'manhattan', 'cityblock'],
          'leaf_size': np.arange(5,40,5), 
          'algorithm' : ['auto']}
grid = GridSearchCV(estimator=KNeighborsClassifier(n_jobs=4), param_grid=params, scoring='f1_weighted')

In [30]:
# fit_and_print(grid)

### Random Forest

In [31]:
params = {'n_estimators': np.arange(40,60,5),
          'max_features': np.arange(2,10),
          'min_samples_split': [2],
          'min_samples_leaf': [2]}
grid = GridSearchCV(estimator=RandomForestClassifier(n_jobs=4), param_grid=params, scoring='f1_weighted')

In [32]:
fit_and_print(grid)

Best estimator:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=55, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy score: 0.553895410886
Classification report
              precision    recall  f1-score   support

        Neg       0.09      0.45      0.14        65
    Neutral       0.83      0.64      0.72       767
        Pos       0.22      0.02      0.04       105

avg / total       0.71      0.55      0.60       937

Confusion matrix
 [[ 29  36   0]
 [272 488   7]
 [ 40  63   2]]


### Neural net

In [33]:
from sklearn.neural_network import MLPClassifier

In [34]:
params = {'alpha': [1e-5],
          'hidden_layer_sizes': [(4,3), (5,3), (8,3), (10,8,3), (10, 5, 3)]}

grid = GridSearchCV(estimator=MLPClassifier(max_iter=2000), param_grid=params, scoring='f1_weighted')

In [35]:
fit_and_print(grid)

Best estimator:
 MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 5, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
Accuracy score: 0.172892209178
Classification report
              precision    recall  f1-score   support

        Neg       0.08      0.80      0.15        65
    Neutral       0.92      0.10      0.18       767
        Pos       0.15      0.30      0.20       105

avg / total       0.77      0.17      0.18       937

Confusion matrix
 [[ 52   2  11]
 [515  78 174]
 [ 68   5  32]]
